# Expected Utility Maximization

Deep dive into portfolio selection via expected utility maximization with different utility functions.

## Utility Functions

1. **Logarithmic**: $U(W) = \log(W)$
   - Equivalent to Kelly criterion
   - Exhibits constant relative risk aversion (CRRA) with γ=1

2. **Power (CRRA)**: $U(W) = \frac{W^{1-\gamma}}{1-\gamma}$
   - γ = relative risk aversion parameter
   - γ > 1: risk averse, γ < 1: risk seeking
   - γ → 1: converges to log utility

3. **Exponential (CARA)**: $U(W) = -e^{-\alpha W}$
   - α = absolute risk aversion
   - Risk aversion independent of wealth level
   - Less common in practice

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from portfolio_gambling.single_period import SinglePeriodOptimizer
from portfolio_gambling.utils import generate_returns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("Expected Utility Maximization")
print("=" * 50)

## 1. Visualize Utility Functions

In [ ]:
# Plot different utility functions
wealth = np.linspace(0.1, 5, 200)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Logarithmic
ax = axes[0, 0]
ax.plot(wealth, np.log(wealth), linewidth=3, color='blue')
ax.set_title('Logarithmic Utility: U(W) = log(W)', fontsize=14, fontweight='bold')
ax.set_xlabel('Wealth (W)', fontsize=12)
ax.set_ylabel('Utility', fontsize=12)
ax.grid(True, alpha=0.3)
ax.axvline(x=1, color='red', linestyle='--', alpha=0.5, label='Initial wealth = 1')
ax.legend()

# Power utility (CRRA) with different γ
ax = axes[0, 1]
gammas = [0.5, 1.0, 2.0, 5.0]
for gamma in gammas:
    if gamma == 1:
        u = np.log(wealth)
    else:
        u = (wealth**(1-gamma) - 1) / (1-gamma)
    ax.plot(wealth, u, linewidth=2, label=f'γ = {gamma}')
ax.set_title('Power Utility (CRRA): U(W) = W^(1-γ)/(1-γ)', fontsize=14, fontweight='bold')
ax.set_xlabel('Wealth (W)', fontsize=12)
ax.set_ylabel('Utility', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend()

# Exponential (CARA)
ax = axes[1, 0]
alphas = [0.5, 1.0, 2.0, 5.0]
for alpha in alphas:
    u = -np.exp(-alpha * wealth)
    ax.plot(wealth, u, linewidth=2, label=f'α = {alpha}')
ax.set_title('Exponential Utility (CARA): U(W) = -exp(-αW)', fontsize=14, fontweight='bold')
ax.set_xlabel('Wealth (W)', fontsize=12)
ax.set_ylabel('Utility', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend()

# Marginal utility comparison
ax = axes[1, 1]
# Log marginal utility: 1/W
ax.plot(wealth, 1/wealth, linewidth=3, label='Log: 1/W', color='blue')
# Power marginal utility: W^(-γ)
ax.plot(wealth, wealth**(-2), linewidth=3, label='Power (γ=2): W^(-2)', color='orange')
# Exponential marginal utility: α*exp(-αW)
ax.plot(wealth, 2*np.exp(-2*wealth), linewidth=3, label='Exponential (α=2): 2exp(-2W)', color='green')
ax.set_title('Marginal Utility: dU/dW', fontsize=14, fontweight='bold')
ax.set_xlabel('Wealth (W)', fontsize=12)
ax.set_ylabel('Marginal Utility', fontsize=12)
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("1. All utility functions are concave (risk averse)")
print("2. Marginal utility decreases with wealth (diminishing returns)")
print("3. Higher γ or α = more risk averse")

## 2. Generate Market Data

In [ ]:
# Create 5 assets
n_assets = 5
n_periods = 1000

mean_returns = np.array([0.08, 0.10, 0.12, 0.15, 0.06]) / 252
volatilities = np.array([0.15, 0.20, 0.25, 0.35, 0.10]) / np.sqrt(252)

corr = np.array([
    [1.0, 0.5, 0.3, 0.2, 0.4],
    [0.5, 1.0, 0.6, 0.4, 0.3],
    [0.3, 0.6, 1.0, 0.7, 0.2],
    [0.2, 0.4, 0.7, 1.0, 0.1],
    [0.4, 0.3, 0.2, 0.1, 1.0]
])

cov_matrix = np.outer(volatilities, volatilities) * corr
returns = generate_returns(n_assets, n_periods, mean_returns, cov_matrix)

optimizer = SinglePeriodOptimizer(returns)

print(f"Generated market data:")
print(f"  Assets: {n_assets}")
print(f"  Periods: {n_periods}")
print(f"  Mean annual returns: {(mean_returns * 252 * 100).round(2)}%")
print(f"  Annual volatilities: {(volatilities * np.sqrt(252) * 100).round(2)}%")

## 3. Optimize for Different Utility Functions

In [ ]:
# Compute optimal portfolios for different utility functions
portfolios = {}

# Log utility (γ=1)
portfolios['Log (γ=1)'] = optimizer.expected_utility('log')

# Power utility with different risk aversions
for gamma in [0.5, 2, 3, 5]:
    portfolios[f'Power (γ={gamma})'] = optimizer.expected_utility('power', gamma=gamma)

# Exponential utility
portfolios['Exponential (α=1)'] = optimizer.expected_utility('exponential', alpha=1.0)
portfolios['Exponential (α=2)'] = optimizer.expected_utility('exponential', alpha=2.0)

# Also compute mean-variance for comparison
portfolios['Mean-Variance'] = optimizer.mean_variance()

# Create DataFrame
weights_df = pd.DataFrame(portfolios, index=[f'Asset {i+1}' for i in range(n_assets)])

print("\nOptimal Portfolio Weights:")
print(weights_df.round(4))
print(f"\nSum of weights:")
print(weights_df.sum().round(4))

## 4. Visualize Portfolio Allocations

In [ ]:
# Stacked bar chart of allocations
fig, ax = plt.subplots(figsize=(14, 8))

weights_df.T.plot(kind='bar', stacked=True, ax=ax, 
                  colormap='tab10', edgecolor='black', linewidth=1.5)

ax.set_title('Portfolio Allocations by Utility Function', fontsize=16, fontweight='bold')
ax.set_xlabel('Utility Function', fontsize=12)
ax.set_ylabel('Portfolio Weight', fontsize=12)
ax.legend(title='Assets', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3, axis='y')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Higher γ (more risk averse) → shift to safer assets")
print("- Log utility (γ=1) ≈ Kelly criterion")
print("- Exponential utility often most conservative")

## 5. Risk-Return Trade-offs

In [ ]:
# Calculate metrics for each portfolio
metrics_list = []
for name, weights in portfolios.items():
    metrics = optimizer.portfolio_metrics(weights)
    metrics['name'] = name
    metrics_list.append(metrics)

metrics_df = pd.DataFrame(metrics_list).set_index('name')

# Annualize
print("\nPortfolio Metrics (Annualized):")
display_df = metrics_df.copy()
display_df['expected_return'] *= 252
display_df['volatility'] *= np.sqrt(252)
display_df['sharpe_ratio'] *= np.sqrt(252)

print(display_df[['expected_return', 'volatility', 'sharpe_ratio']].round(4))

## 6. Certainty Equivalent Analysis

The certainty equivalent is the guaranteed amount that gives the same utility as the risky portfolio:
$$U(CE) = E[U(W)]$$

In [ ]:
def certainty_equivalent(returns, weights, utility_type='log', gamma=2.0, alpha=1.0, initial_wealth=1.0):
    """
    Calculate certainty equivalent for a portfolio.
    """
    portfolio_returns = returns @ weights
    terminal_wealth = initial_wealth * (1 + portfolio_returns)
    
    if utility_type == 'log':
        expected_util = np.mean(np.log(terminal_wealth))
        ce = np.exp(expected_util)
    elif utility_type == 'power':
        if gamma == 1:
            expected_util = np.mean(np.log(terminal_wealth))
            ce = np.exp(expected_util)
        else:
            expected_util = np.mean((terminal_wealth**(1-gamma)) / (1-gamma))
            ce = ((1-gamma) * expected_util) ** (1/(1-gamma))
    elif utility_type == 'exponential':
        expected_util = np.mean(-np.exp(-alpha * terminal_wealth))
        ce = -np.log(-expected_util) / alpha
    
    return ce

# Calculate certainty equivalents
print("\nCertainty Equivalents (starting with $1):")
print("=" * 60)

ce_results = {}
for name, weights in portfolios.items():
    if 'Power' in name:
        gamma = float(name.split('=')[1].replace(')', ''))
        ce = certainty_equivalent(returns, weights, 'power', gamma=gamma)
    elif 'Log' in name:
        ce = certainty_equivalent(returns, weights, 'log')
    elif 'Exponential' in name:
        alpha = float(name.split('=')[1].replace(')', ''))
        ce = certainty_equivalent(returns, weights, 'exponential', alpha=alpha)
    else:
        # Mean-variance: use log utility for comparison
        ce = certainty_equivalent(returns, weights, 'log')
    
    ce_results[name] = ce
    print(f"{name:25s}: ${ce:.4f} (return: {(ce-1)*100:.2f}%)")

print("\nHigher CE = Better portfolio for that utility function")

## 7. Risk Aversion Spectrum

How do optimal portfolios change with risk aversion parameter γ?

In [ ]:
# Sweep across risk aversion parameters
gammas = np.linspace(0.1, 10, 30)
allocations = np.zeros((len(gammas), n_assets))
expected_returns = np.zeros(len(gammas))
volatilities = np.zeros(len(gammas))

for i, gamma in enumerate(gammas):
    w = optimizer.expected_utility('power', gamma=gamma)
    allocations[i] = w
    expected_returns[i] = w @ optimizer.mean_returns
    volatilities[i] = np.sqrt(w @ optimizer.cov_matrix @ w)

# Plot allocation changes
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Allocation vs gamma
ax = axes[0]
for j in range(n_assets):
    ax.plot(gammas, allocations[:, j], linewidth=2.5, label=f'Asset {j+1}', marker='o', markersize=4)
ax.set_xlabel('Risk Aversion (γ)', fontsize=12)
ax.set_ylabel('Portfolio Weight', fontsize=12)
ax.set_title('Portfolio Allocation vs Risk Aversion', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Risk-return vs gamma
ax = axes[1]
ax.plot(gammas, expected_returns * 252, 'b-', linewidth=3, label='Expected Return', marker='s')
ax2 = ax.twinx()
ax2.plot(gammas, volatilities * np.sqrt(252), 'r-', linewidth=3, label='Volatility', marker='o')
ax.set_xlabel('Risk Aversion (γ)', fontsize=12)
ax.set_ylabel('Expected Return (Annualized)', fontsize=12, color='b')
ax2.set_ylabel('Volatility (Annualized)', fontsize=12, color='r')
ax.set_title('Risk-Return Profile vs Risk Aversion', fontsize=14, fontweight='bold')
ax.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='r')
ax.grid(True, alpha=0.3)

lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.tight_layout()
plt.show()

print("\nAs γ increases (more risk averse):")
print("  → Shift to lower-risk assets")
print("  → Lower expected return")
print("  → Lower volatility")

## 8. Comparison with Mean-Variance

Mean-variance is equivalent to quadratic utility OR normal returns with any utility.
Let's see how different utility functions deviate from mean-variance.

In [ ]:
# Plot in mean-std space
plt.figure(figsize=(12, 8))

for name, weights in portfolios.items():
    ret = weights @ optimizer.mean_returns * 252
    vol = np.sqrt(weights @ optimizer.cov_matrix @ weights) * np.sqrt(252)
    
    if name == 'Mean-Variance':
        plt.scatter(vol, ret, s=400, marker='*', c='red', edgecolors='black', 
                   linewidth=2, label=name, zorder=10)
    elif 'Log' in name:
        plt.scatter(vol, ret, s=300, marker='o', c='blue', edgecolors='black',
                   linewidth=2, label=name, zorder=9)
    elif 'Power' in name:
        plt.scatter(vol, ret, s=200, marker='s', edgecolors='black',
                   linewidth=1.5, label=name, alpha=0.7)
    else:
        plt.scatter(vol, ret, s=150, marker='^', edgecolors='black',
                   linewidth=1.5, label=name, alpha=0.7)

plt.xlabel('Volatility (Annualized)', fontsize=12)
plt.ylabel('Expected Return (Annualized)', fontsize=12)
plt.title('Optimal Portfolios: Different Utility Functions', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey insights:")
print("1. Log utility ≈ Mean-variance with moderate risk aversion")
print("2. Higher γ → move left (lower risk)")
print("3. All utility functions lie near the efficient frontier")
print("4. CARA (exponential) often most conservative")

## 9. Practical Considerations

### Choosing a Utility Function:

**Log Utility** (γ=1):
- Good default choice
- Equivalent to Kelly criterion
- Maximizes long-run growth
- Wealth-independent risk aversion

**Power Utility** (γ ≠ 1):
- Flexible risk aversion
- γ = 2-5 common for risk-averse investors
- γ < 1 for risk-seeking (rare)
- Constant relative risk aversion

**Exponential Utility**:
- Constant absolute risk aversion
- Less common in practice
- Can be very conservative
- Risk aversion doesn't scale with wealth

### Estimation Challenges:
1. Need full distribution of returns (not just mean/variance)
2. Sensitive to tail events
3. Computational complexity (Monte Carlo or integration)
4. Parameter estimation (γ, α) from investor behavior

## Key Takeaways

1. **Different utility functions → different optimal portfolios**

2. **Log utility = Kelly criterion** - special case with nice properties

3. **Risk aversion matters** - higher γ → safer portfolios

4. **Certainty equivalent** - useful for comparing portfolios across utilities

5. **Mean-variance approximation** - works well for log utility with γ≈1-2

6. **Practical choice** - log or power with γ=2-3 for most investors

7. **Beyond mean-variance** - utility functions account for full distribution, not just first two moments